In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Carregando os dados
df_breastcancer = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AM/lista-2/breastcancer.csv")

# Dividindo os dados em características (X) e rótulo (y)
X = df_breastcancer.iloc[:, :-1].values  # 30 primeiras colunas são características
y = df_breastcancer.iloc[:, -1].values.reshape(-1, 1)  # Última coluna é a saída

# Normalizando os dados de entrada
X_normalized = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))


In [ ]:
# Calculando média e desvio padrão
def mean_std(values):
    return np.mean(values), np.std(values)

In [ ]:
def gaussian_discriminant_analysis(X_train, y_train):
    # Separar os dados por classe
    X_pos = X_train[y_train[:, 0] == 1]
    X_neg = X_train[y_train[:, 0] == 0]

    # Calcular as médias para cada classe
    mu_pos = np.mean(X_pos, axis=0)
    mu_neg = np.mean(X_neg, axis=0)

    # Calcular as matrizes de covariância para cada classe
    cov_pos = np.cov(X_pos, rowvar=False)
    cov_neg = np.cov(X_neg, rowvar=False)

    # Calcular a matriz de covariância comum
    cov_shared = (len(X_pos) / len(X_train)) * cov_pos + (len(X_neg) / len(X_train)) * cov_neg

    # Calcular os termos constantes para a função discriminante
    inv_cov_shared = np.linalg.inv(cov_shared)
    det_cov_shared = np.linalg.det(cov_shared)
    const_pos = -0.5 * np.log(det_cov_shared) + np.log(len(X_pos) / len(X_train))
    const_neg = -0.5 * np.log(det_cov_shared) + np.log(len(X_neg) / len(X_train))

    return mu_pos, mu_neg, inv_cov_shared, const_pos, const_neg


In [ ]:
def predict_gda(X, mu_pos, mu_neg, inv_cov_shared, const_pos, const_neg):
    # Calcular as funções discriminantes para cada classe
    g_pos = np.sum((X - mu_pos) @ inv_cov_shared * (X - mu_pos), axis=1) + const_pos
    g_neg = np.sum((X - mu_neg) @ inv_cov_shared * (X - mu_neg), axis=1) + const_neg

    # Comparar as funções discriminantes e fazer as previsões
    y_pred = (g_pos < g_neg).astype(int)

    return y_pred

In [ ]:
def evaluate_model_gda(X_train, y_train, X_test, y_test):
    # Treinando o modelo GDA
    mu_pos, mu_neg, inv_cov_shared, const_pos, const_neg = gaussian_discriminant_analysis(X_train, y_train)

    # Fazendo previsões no conjunto de teste
    y_pred = predict_gda(X_test, mu_pos, mu_neg, inv_cov_shared, const_pos, const_neg)

    # Calculando acurácia global
    accuracy = np.mean(y_pred == y_test)

    # Calculando a acurácia por classe
    class_accuracy_0 = np.mean((y_pred == y_test)[y_test[:, 0] == 0])
    class_accuracy_1 = np.mean((y_pred == y_test)[y_test[:, 0] == 1])

    return accuracy, class_accuracy_0, class_accuracy_1


In [ ]:
# Definindo Hiperparâmetros
learning_rate = 0.1
iterations = 100
n_folds = 10
fold_size = len(X) // n_folds

# Inicializando as listas para armazenar as métricas de avaliação
global_accuracy_list_gda = []
class_accuracy_0_list_gda = []
class_accuracy_1_list_gda = []


# Executando a validação cruzada
for i in range(n_folds):
    start = i * fold_size
    end = (i + 1) * fold_size
    X_test = X_normalized[start:end]
    y_test = y[start:end]
    X_train = np.concatenate([X_normalized[:start], X_normalized[end:]])
    y_train = np.concatenate([y[:start], y[end:]])

    # Avaliando o modelo
    global_accuracy, class_accuracy_0, class_accuracy_1 = evaluate_model_gda(X_train, y_train, X_test, y_test)

    # Armazenando as métricas de avaliação
    global_accuracy_list_gda.append(global_accuracy)
    class_accuracy_0_list_gda.append(class_accuracy_0)
    class_accuracy_1_list_gda.append(class_accuracy_1)

# Calculando a média e o desvio padrão das métricas de avaliação
mean_global_accuracy_gda, std_global_accuracy_gda = mean_std(global_accuracy_list_gda)
mean_class_accuracy_0_gda, std_class_accuracy_0_gda = mean_std(class_accuracy_0_list_gda)
mean_class_accuracy_1_gda, std_class_accuracy_1_gda = mean_std(class_accuracy_1_list_gda)

# Imprimindo os resultados
print("Análise do discriminante Gaussiano:")
print("Média da acurácia global:", mean_global_accuracy_gda)
print("Desvio padrão da acurácia global:", std_global_accuracy_gda)
print("Média da acurácia para classe 0:", mean_class_accuracy_0_gda)
print("Desvio padrão da acurácia para classe 0:", std_class_accuracy_0_gda)
print("Média da acurácia para classe 1:", mean_class_accuracy_1_gda)
print("Desvio padrão da acurácia para classe 1:", std_class_accuracy_1_gda)